In [1]:
"""
Project Aegis: Predictive Public Space Conflict Mitigation System
Gradio Interface - Colab & Local Deployment Ready

Installation Instructions for Google Colab:
!pip install gradio folium sklearn pandas numpy

To run in Colab:
Just run the script - Gradio will automatically create a public URL
"""

import gradio as gr
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import folium
from folium import Circle, Marker, Icon, Popup
import json

# Mock ML Model for Risk Score Prediction
class AegisRiskPredictor:
    def __init__(self):
        self.scaler = StandardScaler()
        self.model = RandomForestClassifier(n_estimators=100, random_state=42)
        self._train_model()

    def _train_model(self):
        np.random.seed(42)
        n_samples = 1000
        X = np.random.rand(n_samples, 5) * 100
        y = (X[:, 0] * 0.3 + X[:, 1] * 0.2 + X[:, 2] * 0.25 +
             X[:, 3] * 0.15 + X[:, 4] * 0.1).astype(int)
        y = np.clip(y, 0, 100)
        X_scaled = self.scaler.fit_transform(X)
        self.model.fit(X_scaled, y)

    def predict_risk(self, footfall, sentiment, complaints, event, infrastructure):
        feature_array = np.array([[footfall, sentiment, complaints, event, infrastructure]])
        scaled_features = self.scaler.transform(feature_array)
        risk_score = self.model.predict(scaled_features)[0]
        return int(np.clip(risk_score, 0, 100))

# Initialize model
model = AegisRiskPredictor()

# Mock data for friction zones
FRICTION_ZONES = [
    {
        "name": "Sadar Bazaar",
        "lat": 17.3850,
        "lng": 78.4867,
        "risk_level": "high",
        "risk_score": 87,
        "conflict_type": "Hawker vs Shopkeeper",
        "predicted_time": "4:00 PM - 6:00 PM Today",
        "factors": [
            "• High footfall detected (2.3x normal)",
            "• Festival crowd expected",
            "• Historical conflict pattern on Fridays",
            "• Negative social media sentiment (+45%)"
        ],
        "recommendations": [
            "1. Deploy 2 beat officers between 3-7 PM",
            "2. Activate mobile dispute resolution team",
            "3. Send SMS alert to registered shopkeepers"
        ]
    },
    {
        "name": "Jubilee Hills Park",
        "lat": 17.4239,
        "lng": 78.4082,
        "risk_level": "medium",
        "risk_score": 62,
        "conflict_type": "RWA vs Public - Noise Complaint",
        "predicted_time": "8:00 PM - 11:00 PM Today",
        "factors": [
            "• Wedding event permit issued",
            "• RWA WhatsApp sentiment negative",
            "• Friday evening - high resident presence",
            "• 3 similar complaints in past 2 months"
        ],
        "recommendations": [
            "1. Send proactive SMS to nearby residents",
            "2. Brief event organizers on noise limits",
            "3. Station officer for compliance check at 9 PM"
        ]
    },
    {
        "name": "Ameerpet Ward 5",
        "lat": 17.4374,
        "lng": 78.4482,
        "risk_level": "critical",
        "risk_score": 93,
        "conflict_type": "Water Shortage - Tanker Exploitation",
        "predicted_time": "Next 24 Hours",
        "factors": [
            "• Scheduled water pipeline maintenance",
            "• Historical complaint spike during maintenance",
            "• Social media chatter about tanker prices (+78%)",
            "• Heat wave warning issued"
        ],
        "recommendations": [
            "1. Proactively deploy municipal water tankers",
            "2. Send SMS with official tanker timings",
            "3. Monitor private tanker pricing",
            "4. Set up complaint hotline"
        ]
    },
    {
        "name": "Banjara Hills Parking Zone",
        "lat": 17.4126,
        "lng": 78.4471,
        "risk_level": "medium",
        "risk_score": 58,
        "conflict_type": "Parking Dispute",
        "predicted_time": "6:00 PM - 9:00 PM Today",
        "factors": [
            "• Shopping mall sale event",
            "• Parking capacity at 85%",
            "• 2 disputes reported last weekend",
            "• No traffic management deployment scheduled"
        ],
        "recommendations": [
            "1. Deploy traffic wardens",
            "2. Activate overflow parking signage",
            "3. Send real-time parking availability updates"
        ]
    },
    {
        "name": "Charminar Market",
        "lat": 17.3616,
        "lng": 78.4747,
        "risk_level": "low",
        "risk_score": 28,
        "conflict_type": "General Monitoring",
        "predicted_time": "Normal Operations",
        "factors": [
            "• Regular footfall patterns",
            "• No special events scheduled",
            "• Positive community sentiment",
            "• No historical issues this week"
        ],
        "recommendations": [
            "1. Continue routine patrol",
            "2. Monitor for changes"
        ]
    }
]

def get_risk_color(risk_level):
    colors = {
        "critical": "red",
        "high": "orange",
        "medium": "yellow",
        "low": "green"
    }
    return colors.get(risk_level, "gray")

def create_heatmap():
    """Create the friction heatmap"""
    m = folium.Map(location=[17.3850, 78.4867], zoom_start=12)

    for zone in FRICTION_ZONES:
        color = get_risk_color(zone['risk_level'])

        radius = {
            "critical": 800,
            "high": 600,
            "medium": 400,
            "low": 250
        }.get(zone['risk_level'], 200)

        popup_html = f"""
        <div style="width: 250px; font-family: Arial;">
            <h3 style="margin: 0 0 10px 0; color: #1e40af;">{zone['name']}</h3>
            <div style="background: {'#dc2626' if zone['risk_level']=='critical' else '#ea580c' if zone['risk_level']=='high' else '#f59e0b' if zone['risk_level']=='medium' else '#22c55e'};
                        color: white; padding: 8px; border-radius: 5px; margin-bottom: 10px; text-align: center;">
                <strong>Risk Score: {zone['risk_score']}</strong>
            </div>
            <p style="margin: 5px 0;"><strong>Type:</strong> {zone['conflict_type']}</p>
            <p style="margin: 5px 0;"><strong>Time:</strong> {zone['predicted_time']}</p>
        </div>
        """

        Circle(
            location=[zone['lat'], zone['lng']],
            radius=radius,
            popup=Popup(popup_html, max_width=300),
            color=color,
            fill=True,
            fillColor=color,
            fillOpacity=0.3,
            weight=2
        ).add_to(m)

        Marker(
            location=[zone['lat'], zone['lng']],
            popup=Popup(popup_html, max_width=300),
            icon=Icon(color=color, icon='info-sign')
        ).add_to(m)

    return m

def get_zone_details(zone_name):
    """Get detailed information about a zone"""
    zone = next((z for z in FRICTION_ZONES if z['name'] == zone_name), None)

    if not zone:
        return "Zone not found", "", ""

    # Create header with risk score
    risk_colors = {
        "critical": "#dc2626",
        "high": "#ea580c",
        "medium": "#f59e0b",
        "low": "#22c55e"
    }

    color = risk_colors.get(zone['risk_level'], "#6b7280")

    header = f"""
    <div style="background: {color}; color: white; padding: 20px; border-radius: 10px; text-align: center; margin-bottom: 20px;">
        <h1 style="margin: 0; font-size: 3rem;">{zone['risk_score']}</h1>
        <h2 style="margin: 10px 0 0 0; text-transform: uppercase;">{zone['risk_level']} RISK</h2>
    </div>

    <h3>📍 {zone['name']}</h3>
    <p><strong>Conflict Type:</strong> {zone['conflict_type']}</p>
    <p><strong>Predicted Time:</strong> {zone['predicted_time']}</p>
    """

    factors = "<h3>📊 Risk Factors</h3>\n" + "\n".join(zone['factors'])
    recommendations = "<h3>🛡️ Recommended Actions</h3>\n" + "\n".join(zone['recommendations'])

    return header, factors, recommendations

def predict_risk_score(footfall, sentiment, complaints, event, infrastructure):
    """Predict risk score using ML model"""
    risk_score = model.predict_risk(footfall, sentiment, complaints, event, infrastructure)

    # Determine risk level
    if risk_score >= 85:
        risk_level = "CRITICAL"
        color = "#dc2626"
    elif risk_score >= 70:
        risk_level = "HIGH"
        color = "#ea580c"
    elif risk_score >= 50:
        risk_level = "MEDIUM"
        color = "#f59e0b"
    else:
        risk_level = "LOW"
        color = "#22c55e"

    result = f"""
    <div style="background: {color}; color: white; padding: 30px; border-radius: 10px; text-align: center;">
        <h1 style="margin: 0; font-size: 4rem;">{risk_score}</h1>
        <h2 style="margin: 10px 0 0 0;">{risk_level} RISK</h2>
        <p style="margin: 10px 0 0 0;">Based on AI analysis of provided factors</p>
    </div>

    <div style="margin-top: 20px; padding: 15px; background: #f8fafc; border-radius: 8px;">
        <h3>Input Factors:</h3>
        <ul>
            <li>Footfall Increase: {footfall}%</li>
            <li>Negative Sentiment: {sentiment}</li>
            <li>Historical Complaints: {complaints}</li>
            <li>Event Impact: {event}</li>
            <li>Infrastructure Stress: {infrastructure}</li>
        </ul>
    </div>
    """

    return result

def get_alerts():
    """Get list of active alerts"""
    alerts_html = "<h2>🚨 Active Alerts</h2>\n"

    for zone in FRICTION_ZONES:
        if zone['risk_level'] in ['critical', 'high']:
            color = "#fee2e2" if zone['risk_level'] == 'critical' else "#ffedd5"
            border = "#dc2626" if zone['risk_level'] == 'critical' else "#ea580c"

            alerts_html += f"""
            <div style="background: {color}; border-left: 4px solid {border}; padding: 15px; margin: 10px 0; border-radius: 5px;">
                <h3 style="margin: 0 0 5px 0; color: #991b1b;">{zone['risk_level'].upper()}: {zone['name']}</h3>
                <p style="margin: 5px 0;">{zone['conflict_type']}</p>
                <p style="margin: 5px 0; font-size: 0.9em; color: #666;">Risk Score: {zone['risk_score']} | {zone['predicted_time']}</p>
            </div>
            """

    return alerts_html

# Create Gradio interface
with gr.Blocks(title="Project Aegis - Conflict Mitigation System", theme=gr.themes.Soft()) as demo:

    gr.Markdown("""
    # 🛡️ Project Aegis
    ## Predictive Public Space Conflict Mitigation System
    **Transforming Governance from Reactive to Predictive**
    """)

    with gr.Tabs():
        # Tab 1: Dashboard Overview
        with gr.Tab("📊 Dashboard"):
            gr.Markdown("### City Statistics")

            with gr.Row():
                gr.Markdown("**Critical Alerts:** 1")
                gr.Markdown("**High Risk:** 1")
                gr.Markdown("**Medium Risk:** 2")
                gr.Markdown("**Total Zones:** 5")

            gr.Markdown("---")

            with gr.Row():
                with gr.Column(scale=2):
                    gr.Markdown("### 🗺️ City Friction Heatmap")
                    map_display = gr.HTML(create_heatmap()._repr_html_())

                    gr.Markdown("""
                    #### Legend
                    🔴 **Critical (85-100)** | 🟠 **High (70-84)** | 🟡 **Medium (50-69)** | 🟢 **Low (0-49)**
                    """)

                with gr.Column(scale=1):
                    alerts = gr.HTML(get_alerts())

        # Tab 2: Zone Details
        with gr.Tab("📍 Zone Details"):
            gr.Markdown("### Select a zone to view detailed analysis")

            zone_dropdown = gr.Dropdown(
                choices=[z['name'] for z in FRICTION_ZONES],
                label="Select Zone",
                value=FRICTION_ZONES[0]['name']
            )

            zone_header = gr.HTML()
            zone_factors = gr.Markdown()
            zone_recommendations = gr.Markdown()

            deploy_btn = gr.Button("🚀 Deploy Intervention Team", variant="primary", size="lg")
            deploy_status = gr.Markdown()

            def deploy_team(zone_name):
                return f"✅ **Intervention team successfully deployed to {zone_name}!**"

            zone_dropdown.change(
                fn=get_zone_details,
                inputs=[zone_dropdown],
                outputs=[zone_header, zone_factors, zone_recommendations]
            )

            deploy_btn.click(
                fn=deploy_team,
                inputs=[zone_dropdown],
                outputs=[deploy_status]
            )

            # Load initial zone
            demo.load(
                fn=get_zone_details,
                inputs=[zone_dropdown],
                outputs=[zone_header, zone_factors, zone_recommendations]
            )

        # Tab 3: Risk Prediction
        with gr.Tab("🤖 AI Risk Predictor"):
            gr.Markdown("""
            ### Simulate Risk Prediction
            Adjust the factors below to see how the AI predicts conflict risk for any location.
            """)

            with gr.Row():
                with gr.Column():
                    footfall_slider = gr.Slider(0, 200, value=50, label="Footfall Increase (%)")
                    sentiment_slider = gr.Slider(0, 100, value=30, label="Negative Sentiment Score")
                    complaints_slider = gr.Slider(0, 100, value=40, label="Historical Complaints")

                with gr.Column():
                    event_slider = gr.Slider(0, 100, value=20, label="Event Impact Score")
                    infrastructure_slider = gr.Slider(0, 100, value=30, label="Infrastructure Stress")

            predict_btn = gr.Button("🔮 Predict Risk Score", variant="primary", size="lg")
            prediction_output = gr.HTML()

            predict_btn.click(
                fn=predict_risk_score,
                inputs=[footfall_slider, sentiment_slider, complaints_slider,
                       event_slider, infrastructure_slider],
                outputs=[prediction_output]
            )

        # Tab 4: About
        with gr.Tab("ℹ️ About"):
            gr.Markdown("""
            ## About Project Aegis

            **Project Aegis** is a predictive public space conflict mitigation system that uses AI and data fusion
            to identify potential conflicts before they occur.

            ### How It Works

            1. **Data Ingestion**: The system securely ingests anonymized data from multiple sources:
               - Historical citizen grievance records
               - Social media sentiment analysis
               - Urban infrastructure and event data
               - Aggregated mobility patterns

            2. **AI Analysis**: Machine learning models analyze correlations across data streams to predict
               conflict hotspots with high accuracy.

            3. **Actionable Intelligence**: The system provides real-time alerts and specific recommendations
               for preventive intervention.

            ### Key Benefits

            - **For Citizens**: Peaceful public spaces and proactive governance
            - **For Police**: Data-driven resource allocation and preventive deployment
            - **For Administration**: Early intervention prevents escalation into major issues

            ### Privacy & Ethics

            Project Aegis is designed with privacy as a core principle:
            - All data is anonymized and aggregated
            - No individual tracking or surveillance
            - Focus on systemic patterns, not individual behavior

            ---

            **Project Aegis** - Empowering proactive governance through predictive analytics
            """)

# Launch the interface
if __name__ == "__main__":
    demo.launch(share=True)  # share=True creates a public URL in Colab

/tmp/ipython-input-2248212727.py:200: UserWarning: color argument of Icon should be one of: {'red', 'blue', 'lightgray', 'gray', 'lightred', 'beige', 'darkblue', 'black', 'white', 'orange', 'pink', 'lightblue', 'cadetblue', 'green', 'purple', 'darkpurple', 'darkred', 'darkgreen', 'lightgreen'}.
  icon=Icon(color=color, icon='info-sign')


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://26299f95ff6085093e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
